In [1]:
import numpy as np
import pandas as pd
import json

import plotly.express as px
import re

import torch
import tensorflow as tf
from sklearn.model_selection import train_test_split

2023-02-23 11:32:21.683467: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-23 11:32:21.683516: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
def get_metadata():
    with open('/data/elastic-notebook/data/arxiv-metadata-oai-snapshot/arxiv-metadata-oai-snapshot.json', 'r') as f:
        for line in f:
            yield line

In [3]:
metadata = get_metadata()

In [4]:
titles_tags_dict = {"title":[], "tags":[]}
for paper in metadata:
    parsed = json.loads(paper)
    titles_tags_dict["title"].append(parsed['title'])
    titles_tags_dict["tags"].append(parsed['categories'])

In [5]:
titles_tags_df = pd.DataFrame.from_records(titles_tags_dict)

In [6]:
titles_tags_df.head(5)

,tags,title
0,hep-ph,Calculation of prompt diphoton production cros...
1,math.CO cs.CG,Sparsity-certifying Graph Decompositions
2,physics.gen-ph,The evolution of the Earth-Moon system based o...
3,math.CO,A determinant of Stirling cycle numbers counts...
4,math.CA math.FA,From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...


## Getting tag label arrays

In [7]:
categories = titles_tags_df['tags'].apply(lambda x: x.split(' ')).explode().unique()

In [8]:
label_to_int_dict = {}
for i, key in enumerate(categories):
    label_to_int_dict[key] = i

In [9]:
int_to_label_dict = {}
for key, val in label_to_int_dict.items():
    int_to_label_dict[val] = key

In [10]:
def generate_label_array(label):
    result = np.zeros(len(label_to_int_dict))
    labels = label.split(' ')
    for l in labels:
        result[label_to_int_dict[l]] = 1
    return np.expand_dims(result, 0)

In [11]:
tag_labels = [generate_label_array(tag) for tag in titles_tags_df["tags"]]

In [12]:
tag_labels = np.concatenate(tag_labels, axis = 0)

In [13]:
tag_labels[1]

array([0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0.])

## Getting tokenized title arrays

In [14]:
titles = titles_tags_df['title'].apply(lambda x : x.lower())
titles = titles.apply(lambda x: re.sub('[^A-Za-z\s]+', ' ', x))
titles = titles.apply(lambda x: re.sub('\n', ' ', x))
titles = titles.apply(lambda x: re.sub(r'/s+', ' ', x))
titles = titles.apply(lambda x: re.sub(r'^/s', '', x))
titles = titles.apply(lambda x: re.sub(r'/s$', '', x))

In [15]:
title_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')

In [16]:
title_tokenizer.fit_on_texts(titles)

In [17]:
title_tokens = title_tokenizer.texts_to_sequences(titles)

In [18]:
title_tokens = tf.keras.preprocessing.sequence.pad_sequences(title_tokens, maxlen=20, padding='post')

In [19]:
title_tokens[1]

array([2540, 8608,  129, 1963,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

## Creating dataset

In [20]:
tag_labels.shape[0] == title_tokens.shape[0]

True

In [21]:
x_train, x_test, y_train, y_test = train_test_split(title_tokens, tag_labels, test_size = 0.2)

In [22]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(len(title_tokenizer.word_index)+1, 128),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(len(categories), activation = 'sigmoid')
])

2023-02-23 11:34:02.028146: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-02-23 11:34:02.028180: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-02-23 11:34:02.028201: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (yongjoo-serv-01): /proc/driver/nvidia/version does not exist
2023-02-23 11:34:02.028406: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [23]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
             optimizer=tf.keras.optimizers.Adam(),
             metrics=['accuracy'])

In [24]:
hist = model.fit(x_train, y_train, batch_size = 128, epochs = 1)

Epoch 1/200
  292/13798 [..............................] - ETA: 40:05 - loss: 0.0952 - accuracy: 0.0447Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/srv/local/data/elastic-notebook/venv/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3378, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_3830387/207946138.py", line 1, in <module>
    hist = model.fit(x_train, y_train, batch_size = 128, epochs = 200)
  File "/srv/local/data/elastic-notebook/venv/lib/python3.8/site-packages/keras/utils/traceback_utils.py", line 64, in error_handler
    return fn(*args, **kwargs)
  File "/srv/local/data/elastic-notebook/venv/lib/python3.8/site-packages/keras/engine/training.py", line 1384, in fit
    tmp_logs = self.train_function(iterator)
  File "/srv/local/data/elastic-notebook/venv/lib/python3.8/site-packages/tensorflow/python/util/traceback_utils.py", line 150, in error_handler
    return fn(*args, **kwargs)
  File "/srv/local/data/elastic-notebook/venv/lib/python3.8/site-packages/tensorflow/python/eager/def_function.py", line 915, in __call__
